In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt   
import pandas as pd
import random
import numpy as np
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
from time import sleep
import scipy
import operator
import difflib
import math
from IPython.core.display import display,HTML
try:
    import cPickle as pickle     #it is faster than pickle!
except:
    import pickle
    
import unicodedata
import networkx as nx
import itertools
import seaborn as sns   ### https://seaborn.pydata.org/tutorial/categorical.html
    
display(HTML("<style>.container { width:100% !important; }</style>"))  # to make the notebook use the entire width of the browser



import plotly.plotly as py
from plotly.graph_objs import *

# i only need my credentials if i want to plot online --- and send plots to server (limits per day apply!)
#import plotly.tools as tls
#tls.set_credentials_file(username='juliettapc', api_key='deyNIvtOoDZ5PLmrHlhd')  # my plotly account credentials


import pygraphviz
from networkx.drawing.nx_agraph import graphviz_layout




########## to be able to plot offline (without sending the plots to the plotly server every time)
import plotly.offline as offline
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

In [ ]:
import plotly
plotly.__version__

In [2]:

path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Data/Dropbox/"
input_file='Dropbox_datafile_may22_2017_modified.csv'
df=pd.read_csv(path+input_file, sep=',',na_values=["NAN","-1","null"],low_memory=False, parse_dates=['folder_creation_date','date_last_change']) # set header=0 if i wanna pass it my own list of header names
df = df.drop('Unnamed: 0', 1)
print df.shape
df.head()

(2592500, 15)


,folder_id,num_folder_members,folder_creation_date,date_last_change,user_id,email_domain,num_adds,num_edits,num_deletes,major_content_type,major_content_ext,group_total_publ,group_num_papers_last,group_num_citations,folder_lifespan
0,592732,4,2008-06-12,2012-12-25,77422,cornell.edu,NaN,NaN,NaN,office_docs,doc,NaN,NaN,NaN,1658
1,592732,4,2008-06-12,2012-12-25,79630,cornell.edu,NaN,NaN,NaN,office_docs,doc,NaN,NaN,NaN,1658
2,592732,4,2008-06-12,2012-12-25,77619,cornell.edu,80.0,72.0,0.0,office_docs,doc,NaN,NaN,NaN,1658
3,604794,2,2008-06-27,2012-12-23,86572,unc.edu,21798.0,40498.0,10660.0,pdf,pdf,NaN,NaN,NaN,1641
4,680152,2,2008-08-26,2013-01-04,76579,cs.princeton.edu,644.0,0.0,112.0,pdf,pdf,NaN,NaN,NaN,1593


In [ ]:
df.shape

In [ ]:
for item in df.email_domain.unique():
    print item,"   "

In [3]:
#### time window for temporal networks
string_start_date='2014-05-01'
start_date=pd.Timestamp(string_start_date)

#string_end_date='2014-11-01'
end_date=start_date + pd.Timedelta('180 days')




list_tupla_dates=[]
for i in range(6):
    print start_date, end_date    
    tupla_dates=[start_date,end_date]
    list_tupla_dates.append(tupla_dates)

    
    start_date += pd.Timedelta('180 days')
    end_date += pd.Timedelta('180 days')

# 2014-05-01_2014-10-28

2014-05-01 00:00:00 2014-10-28 00:00:00
2014-10-28 00:00:00 2015-04-26 00:00:00
2015-04-26 00:00:00 2015-10-23 00:00:00
2015-10-23 00:00:00 2016-04-20 00:00:00
2016-04-20 00:00:00 2016-10-17 00:00:00
2016-10-17 00:00:00 2017-04-15 00:00:00


In [ ]:

### for plotting time series directly from data
#df.plot(x='folder_creation_date',y='folder_lifespan',ls='',marker='.') 



In [34]:
list_tupla_dates

[[Timestamp('2014-10-28 00:00:00'), Timestamp('2015-04-26 00:00:00')],
 [Timestamp('2015-04-26 00:00:00'), Timestamp('2015-10-23 00:00:00')],
 [Timestamp('2015-10-23 00:00:00'), Timestamp('2016-04-20 00:00:00')],
 [Timestamp('2016-04-20 00:00:00'), Timestamp('2016-10-17 00:00:00')],
 [Timestamp('2016-10-17 00:00:00'), Timestamp('2017-04-15 00:00:00')],
 [Timestamp('2017-04-15 00:00:00'), Timestamp('2017-10-12 00:00:00')]]

In [ ]:
#print df.folder_creation_date.min(), df.folder_creation_date.max()   ###  2008-05-01 00:00:00     2017-05-16 00:00:00
#print df.date_last_change.min(), df.date_last_change.max()   ###          2012-11-07 00:00:00     2017-05-17 00:00:00


In [ ]:
df[df.user_id == 86572]

#table[table.column_name == some_value]

In [4]:
flag_GC="yes"  # yes: only plot the GC;    no: i plot the entire network,   m: i select multiple big connected components for plotting

Num_con_comp=10


#focus_university="harvard.edu"           #### harvard   missouri      northwestern   berkeley  ucla.  cambridge    ox.ac.uk
# list_focus_univ=["missouri.edu","harvard.edu","northwestern.edu", "berkeley","ucla.","cambridge","ox.ac.uk","ucdavis","princeton","uchicago","umass","columbia","unizar.es","duke.edu"]
list_focus_univ=["northwestern.edu"]
# list_focus_univ=["berkeley"]


for tupla_dates in tqdm_notebook(list_tupla_dates):
    


    start_date=tupla_dates[0]
    end_date=tupla_dates[1]

    print "buiding network for time window:",str(start_date).replace("00:00:00","").strip(),"--",str( end_date).replace("00:00:00","").strip()

    
    G=nx.Graph()

    dict_folder_list_domains={}
    dict_folder_list_users={}
    dict_user_id_domain={}
    dict_user_id_edit={}
    dict_user_id_del={}
    dict_user_id_add={}
    dict_user_id_folder_lifespan={}



    print df.shape
    cont=0


    for row in tqdm_notebook(df.iterrows()):

        folder=row[1]["folder_id"]
        user_id=row[1]["user_id"]                              
        email=row[1]["email_domain"]    
        edits=row[1]["num_edits"]
        adds=row[1]["num_adds"]
        deletes=row[1]["num_deletes"]   
        folder_lifespan=row[1]["folder_lifespan"]
        num_members=row[1]["num_folder_members"]


        folder_creation_date=row[1]["folder_creation_date"]
        date_last_change=row[1]["date_last_change"]


        #     for focus_university in list_focus_univ:                 
    #         if focus_university in email:
    #             flag_useful_line =1
    #             break


        flag_useful_line=0    
        if (folder_creation_date >= start_date  and folder_creation_date <= end_date)     or     ( date_last_change >= start_date  and  date_last_change <= end_date ):
            flag_useful_line =1



        if flag_useful_line ==1:
                try: 
                    dict_folder_list_users[folder].append(user_id)
                except:
                    dict_folder_list_users[folder]=[]
                    dict_folder_list_users[folder].append(user_id)

                try:    
                    dict_folder_list_domains[folder].append(email)
                except:            
                    dict_folder_list_domains[folder]=[]
                    dict_folder_list_domains[folder].append(email)



                dict_user_id_domain[user_id]=email   

                try:
                    dict_user_id_edit[user_id] +=edits   # ojo!! the records are by folder and by user, so i need to accumulate!
                    dict_user_id_del[user_id] +=deletes
                    dict_user_id_add[user_id] +=adds 

                    dict_user_id_folder_lifespan[user_id].append(folder_lifespan) 

                except :
                    dict_user_id_edit[user_id] =edits
                    dict_user_id_del[user_id] =deletes
                    dict_user_id_add[user_id] =adds 

                    dict_user_id_folder_lifespan[user_id]=[]
                    dict_user_id_folder_lifespan[user_id].append(folder_lifespan) 



    #     cont +=1
    #     if cont % 100000 ==0:
    #         print cont

    print "done with the dict., size:",len(dict_folder_list_users)





    for user in  dict_user_id_folder_lifespan:
        dict_user_id_folder_lifespan[user]=np.median(dict_user_id_folder_lifespan[user])



    for folder in dict_folder_list_users:
        dict_folder_list_users[folder]=list(set(dict_folder_list_users[folder]))  # remove possible duplicates

        dict_folder_list_domains[folder]=list(set(dict_folder_list_domains[folder]))


        lista=dict_folder_list_users[folder]

        if len(lista)>1:

            lista_pares=itertools.combinations(lista, 2)        

            for item in lista_pares:
                e1=item[0]
                e2=item[1]
                G.add_edge(e1,e2)
               # print "added edge:",e1, e2
    #     else:   # FOR NOW, I DO NOT ADD THE NODES WITH COLLABORATORS OUTSIDE THE UNIVERSITY DOMAINS
    #         user_id=lista[0]

    #         G.add_node(user_id)



    print "  N:", len(G.nodes()),"  L:", len(G.edges())    





    GC_string=""
    if flag_GC =="yes":   # i will plot only the GC
        print "selecting only the GC for plotting"
        G = max(nx.connected_component_subgraphs(G), key=len)
        print "  N:", len(G.nodes()),"  L:", len(G.edges())    
        GC_string="_GC"

    elif flag_GC =="m":  # i will plot multiple large connected components

        print "selecting GC and some other large connected components for plotting"

        new_G = max(nx.connected_component_subgraphs(G), key=len)
        print "  N:",len(new_G.nodes()),"  L:",len(new_G.edges())
        lista_comp= reversed(sorted(nx.connected_component_subgraphs(G), key=len))


        cont_comp=0
        for comp in lista_comp:
    #         cont_comp +=1
    #         new_G.add_nodes_from(comp.nodes())
    #         new_G.add_edges_from(comp.edges())
    #         print cont_comp,"  N:",len(new_G.nodes()),"  L:",len(new_G.edges())
    #         if cont_comp >=Num_con_comp:
    #             break


            if len(comp.nodes())>=10:
                new_G.add_nodes_from(comp.nodes())
                new_G.add_edges_from(comp.edges())
                print "  N:",len(new_G.nodes()),"  L:",len(new_G.edges())




        G= new_G.copy()
        GC_string="_multiple_comp"


    elif flag_GC =="no":  # i will plot the entire structure
        print "selecting entire network for plotting"
       # print focus_university, "done.  N:", len(G.nodes()),"  L:", len(G.edges())
        GC = max(nx.connected_component_subgraphs(G), key=len)
        print "\n  GC:     N:", len(GC.nodes()), "  L:", len(GC.edges())





    #### the master dict is larger than G.nodes(), and the set_node_attribute does not work in that case, so i get a selection of the dict only with the common nodes:
    #######################
    print len(G.nodes()), len(dict_user_id_domain)

    new_dict_user_id = {k: dict_user_id_domain[k] for k in set(G.nodes()) & set(dict_user_id_domain.keys())}
    print len(G.nodes()), len(new_dict_user_id)
    # i add the email as node attributes    
    nx.set_node_attributes(G, 'email', new_dict_user_id)
    ############



    new_dict_user_id = {k: dict_user_id_edit[k] for k in set(G.nodes()) & set(dict_user_id_edit.keys())}
    # i add the number of edits as node attributes    
    nx.set_node_attributes(G, 'edits', new_dict_user_id)
    ############



    new_dict_user_id = {k: dict_user_id_del[k] for k in set(G.nodes()) & set(dict_user_id_del.keys())}
    # i add the number of deletes as node attributes    
    nx.set_node_attributes(G, 'deletes', new_dict_user_id)
    ############



    new_dict_user_id = {k: dict_user_id_add[k] for k in set(G.nodes()) & set(dict_user_id_add.keys())}
    # i add the number of additions as node attributes    
    nx.set_node_attributes(G, 'adds', new_dict_user_id)
    ############



    new_dict_user_id = {k: dict_user_id_folder_lifespan[k] for k in set(G.nodes()) & set(dict_user_id_folder_lifespan.keys())}
    # i add the median folder_lifespan as node attributes 
    nx.set_node_attributes(G, 'median_folder_lifespan', new_dict_user_id)
    ############











    path="/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/Networks/"
    filename="first_network_"+str(start_date).replace("00:00:00","").strip()+"_"+str(end_date).replace("00:00:00","").strip()+".pickle"
    with open(path+filename,'wb') as f:
        pickle.dump(G, f)
    print "written:",path+filename






buiding network for time window: 2014-05-01 -- 2014-10-28
(2592500, 15)



KeyboardInterrupt: 

In [5]:
###### I get a different node shape for each domain (and i group all subdomains together)
dict_domains_cont={}
cont=0
for node in G.nodes():    
    
    domain=str(G.node[node]['email'])
    for focus_university in list_focus_univ:   
        if focus_university in domain:
            domain=focus_university
            break
    
    if domain in dict_domains_cont:
        G.node[node]['shape']=dict_domains_cont[domain]
    else:
        cont +=1
        dict_domains_cont[domain]=cont
        G.node[node]['shape']=dict_domains_cont[domain]


        

print "domains:",set(dict_domains_cont.keys()),"   shapes",set(dict_domains_cont.values())

domains: set([])    shapes set([])
1135617it [05:28, 3461.26it/s]

In [7]:

########### i relabel the nodes so the plotly works later (otherwise, it doesnt work!)
mapping={}
cont=0
for n in G.nodes():
    mapping[n]=cont
    cont +=1

G=nx.relabel_nodes(G,mapping)
print "nodes relabeled"
##############################

    
    

##### i get a layout for the plotting of the network
layt={}

layt=nx.fruchterman_reingold_layout(G,dim=3)     
#dict_pos=nx.fruchterman_reingold_layout(G,dim=2)
#dict_pos=nx.spectral_layout(G,dim=2)
#dict_pos=nx.spring_layout(G, dim=2)


    ##### layout options for   graphviz_layout:
    # dot - "hierarchical" or layered drawings of directed graphs. This is the default tool to use if edges have directionality.
    # neato - "spring model'' layouts.  This is the default tool to use if the graph is not too large (about 100 nodes) and you don't know anything else about it. Neato attempts to minimize a global energy function, which is equivalent to statistical multi-dimensional scaling.
    # fdp - "spring model'' layouts similar to those of neato, but does this by reducing forces rather than working with energy.
    # sfdp - multiscale version of fdp for the layout of stanfordlarge graphs.
    # twopi - radial layouts, after Graham Wills 97. Nodes are placed on concentric circles depending their distance from a given root node.
#     circo - circular layout, after Six and Tollis 99, Kauffman and Wiese 02. This is suitable for certain diagrams of multiple cyclic structures, such as certain telecommunications networks.

# layt=graphviz_layout(G, prog='fdp')

#### i save the resulting  layout as attributes of the nodes themselves
nx.set_node_attributes(G, 'pos', layt)

print "done getting the postion dict for",len(layt),"nodes"


nodes relabeled
done getting the postion dict for 0 nodes


In [ ]:
colors_as="folder_lifespan"  ## or "edits"  or folder_lifespan or k

##### i create the list for the info that will show up when hovering over a node, as well as for giving each node a color-coded, shape and size
list_values_colors=[]
list_labels=[]
list_sizes=[]
list_symols=[]
for node in G.nodes():
    if colors_as =="k":
        list_values_colors.append(G.degree(node))
        title_bar="degree"
    elif colors_as == "edits" :
        list_values_colors.append(G.node[node]['edits'])
        title_bar="number edits"
    elif colors_as == "folder_lifespan" :
        list_values_colors.append(G.node[node]['median_folder_lifespan'])
        title_bar="median lifespan of user's folders"
        
         
        
        
        
    
    #list_sizes.append(random.choice([1,2,3,4,5,6,7,8,9,10]))
    list_sizes.append(int(math.log(G.node[node]['edits']+0.001)+1))

    list_symols.append(G.node[node]['shape'])
                                  
    label=G.node[node]['email']+"  degree: "+str(G.degree(node))+"<br>ed: "+ str(G.node[node]['edits'])+"  adds: "+ str(G.node[node]['adds'])+"  del: "+ str(G.node[node]['deletes'])+"<br>median lifespan: "+str(G.node[node]['median_folder_lifespan'])
    list_labels.append(label)

    
    ## for the label, if i want a new line, code it as:   <br>  not as  \n
    
    # see complete list of symbols for nodes at:         https://plot.ly/matlab/reference/#scatter-marker  and    https://plot.ly/~neda/1032/hover-over-the-marker-points-to-see-their-names/
   

In [ ]:
####### Set data for the Plotly plot of the graph:
if len(list_focus_univ)==1:
    plot_title=str(list_focus_univ).replace("[","").replace("]","").replace(",","-").replace('"','').title()
else:
    plot_title="multiple universities"


N=len(G.nodes())
L=len(G.edges())

Xn=[layt[k][0] for k in range(N)]# x-coordinates of nodes
Yn=[layt[k][1] for k in range(N)]# y-coordinates
Zn=[layt[k][2] for k in range(N)]# z-coordinates
Xe=[]
Ye=[]
Ze=[]
for e in G.edges():
    Xe+=[layt[e[0]][0],layt[e[1]][0], None]# x-coordinates of edge ends
    Ye+=[layt[e[0]][1],layt[e[1]][1], None]
    Ze+=[layt[e[0]][2],layt[e[1]][2], None]
    
    
    
    
    
trace1=Scatter3d(x=Xe,
               y=Ye,
               z=Ze,
               mode='lines',
               line=Line(color='rgb(125,125,125)', width=1),
               hoverinfo='none'
               )
trace2=Scatter3d(x=Xn,
               y=Yn,
               z=Zn,
               mode='markers',
               name='actors',
               marker=Marker(symbol='dot',
                             size=list_sizes, #6,
                             color=list_values_colors,
                             colorscale='Viridis',
                             line=Line(color='rgb(50,50,50)', width=0.5),                                                                                       
                             colorbar=dict(
                                thickness=15,
                                title=title_bar,#title='degree',
                                xanchor='left',
                                titleside='right'),                                
                             ),
               text=list_labels,
               hoverinfo='text'
               )
    
    
    
    
axis=dict(showbackground=False,
          showline=False,
          zeroline=False,
          showgrid=False,
          showticklabels=False,
          title=''
          )





layout = Layout(
         title=plot_title,
         width=1000,
         height=1000,
         showlegend=False,
         scene=Scene(
         xaxis=XAxis(axis),
         yaxis=YAxis(axis),
         zaxis=ZAxis(axis),
        ),
     margin=Margin(
        t=100
    ),
    hovermode='closest',
    annotations=Annotations([
           Annotation(
           showarrow=False,
            text="Data source: Dropbox",
            xref='paper',
            yref='paper',
            x=0,
            y=0.1,
            xanchor='left',
            yanchor='bottom',
            font=Font(
            size=14
            )
            )
        ]),    )

print "done"

In [ ]:
print "N:", len(G.nodes()), "L:", len(G.edges())

In [ ]:
data=Data([trace1, trace2])
fig=Figure(data=data, layout=layout)

iplot(fig)

In [ ]:
# save the image as png and as html (interactive)

filename=str(list_focus_univ).replace("[","").replace("]","").replace(",","-").replace('"','')+GC_string
#py.image.save_as(fig, filename='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/'+filename+'_color_'+colors_as+GC_string+'_3D_.png',scale=5)  #scale controls the resolution of the image

offline.plot(fig, filename='/home/juliaponcela/at_NICO/Dropbox_collaboration_patterns/Results/'+filename+'_color_'+colors_as+GC_string+'_3D_.html')



In [ ]:
# help(py.image)

In [ ]:
# with open(path+"first_network_"+focus_university+".pickle",'wb') as f:
#     %time    pickle.dump(G, f)
# print "written:",path+"first_network_"+focus_university+".pickle\n"




# with open(path+"dict_position_first_network_"+focus_university+".pickle",'wb') as f:
#     %time    pickle.dump(dict_pos, f)
# print "written:",path+"dict_position_first_network_"+focus_university+".pickle"


In [ ]:
import plotly.plotly as py
import pandas as pd